# Memory

In [130]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        super().__init__()
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [132]:
store = {} # item(key=session_id, value=InMemoryHistory 인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [133]:
from langchain_core.messages import HumanMessage
history_test = get_by_session_id('user1')
# history_test.add_message(HumanMessage(content='저녁 뭐 먹지?'))

print(store)

{'user1': []}


In [134]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

# prompt = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_template('너는 {skill}을 잘하는 AI 어시스턴트야.'),
#     MessagesPlaceholder(variable_name='history'),
#     HumanMessagePromptTemplate.from_template('{query}')
# ])


prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [135]:
response = chain_with_history.invoke(
    input={'skill': '대화', 'query': '토끼는 나무 두 그루를 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='재미있는 설정이네요! 토끼가 나무를 키우는 이유나 그 나무에 대한 이야기를 더 해주실 수 있나요? 어떤 종류의 나무인지, 토끼가 어떻게 돌보고 있는지 궁금합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 40, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ5SqljdrBk67xjg6H0yaYJRd90CW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--0778cad3-2bdf-42cb-b427-ab4d01fa7e96-0' usage_metadata={'input_tokens': 40, 'output_tokens': 52, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [136]:
response = chain_with_history.invoke(
    input={'skill': '대화', 'query': '다람쥐는 도토리 나무 세 그루를 키우고 있습니다.'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 도토리 나무 세 그루를 키우고 있다니, 정말 흥미로운 상황이네요! 다람쥐는 도토리를 좋아하는 동물이라 도토리 나무를 키우는 것은 그들에게 아주 유익할 것 같아요. 도토리 나무는 나무가 성장하는 과정에서 어떤 변화가 있을까요? 또는 다람쥐가 이 나무들을 어떻게 관리할까요? 더 이야기해 볼까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 46, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ5SrgrmC9tDWXa7M1WZuGfC5qWLj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--c821b5be-dc70-4fdd-bc12-76dac1afb635-0' usage_metadata={'input_tokens': 46, 'output_tokens': 105, 'total_tokens': 151, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [137]:
store['rabbit']

[HumanMessage(content='토끼는 나무 두 그루를 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='재미있는 설정이네요! 토끼가 나무를 키우는 이유나 그 나무에 대한 이야기를 더 해주실 수 있나요? 어떤 종류의 나무인지, 토끼가 어떻게 돌보고 있는지 궁금합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 40, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ5SqljdrBk67xjg6H0yaYJRd90CW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--0778cad3-2bdf-42cb-b427-ab4d01fa7e96-0', usage_metadata={'input_tokens': 40, 'output_tokens': 52, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [138]:
response = chain_with_history.invoke(
    input={'skill': '대화', 'query': '토끼는 나무 두 그루를 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 나무를 두 그루 키우고 있다니, 정말 귀여운 상상이네요! 혹시 그 나무들이 어떤 종류인지, 토끼가 어떻게 돌보고 있는지에 대해 더 이야기해 주실 수 있나요? 또는 이 이야기를 바탕으로 어떤 내용을 더 발전시켜보고 싶으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 114, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ5Su1YrMT2d1wSLGHQk7T1D5UBDY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--360ec920-e4f6-4889-8df7-e1ddbfc105d6-0' usage_metadata={'input_tokens': 114, 'output_tokens': 72, 'total_tokens': 186, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [139]:
response = chain_with_history.invoke(
    input={'skill': '대화', 'query': '토끼와 다람쥐는 총 몇그루의 나무를 키우고 있나요?'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 두 그루의 나무를 키우고 있다면, 다람쥐가 나무를 키우고 있지 않다면 총 두 그루의 나무가 있습니다. 하지만 다람쥐도 나무를 키우고 있다면, 그 수에 따라 총 나무의 개수가 달라질 수 있습니다. 다람쥐가 몇 그루의 나무를 키우고 있는지 알려주시면 더 정확한 답을 드릴 수 있을 것 같아요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 217, 'total_tokens': 325, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ5SwznIhDR1TckkGd8MlMb2P9Gdk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--f657885f-d305-4894-a9b2-d9972a0e38e5-0' usage_metadata={'input_tokens': 217, 'output_tokens': 108, 'total_tokens': 325, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [140]:
store

{'user1': [],
 'rabbit': [HumanMessage(content='토끼는 나무 두 그루를 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='재미있는 설정이네요! 토끼가 나무를 키우는 이유나 그 나무에 대한 이야기를 더 해주실 수 있나요? 어떤 종류의 나무인지, 토끼가 어떻게 돌보고 있는지 궁금합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 40, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ5SqljdrBk67xjg6H0yaYJRd90CW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--0778cad3-2bdf-42cb-b427-ab4d01fa7e96-0', usage_metadata={'input_tokens': 40, 'output_tokens': 52, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de